In [105]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import importlib
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from definitions import ROOT_DIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
from src.features.multi_omics import MultiOmicsData
# importlib.reload(src.features.multi_omics)

import pandas as pd
import numpy as np

In [163]:
folder_path = ROOT_DIR + "/data/tcga-assembler/LUAD/"
luad_data = MultiOmicsData(cancer_type="LUAD", folder_path=folder_path,
                           modalities=["GE", "MIR", "LNC"])
matched_samples = luad_data.match_samples(modalities=["GE", "MIR"])

/opt/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


PATIENTS (522, 5)
BIOSPECIMENS (126, 2)
DRUGS (461, 4)
GE (576, 20472)
MIR (494, 1870)
LNC (546, 12727)


In [164]:
X_multiomics, y = luad_data.load_data('all')
print(X_multiomics['GE'].shape, X_multiomics['MIR'].shape, X_multiomics['LNC'].shape, y.shape)
y['ajcc_pathologic_tumor_stage'].value_counts()

modalities matched sample size: (433, 1)
joined clinical data size: (433, 6)
(429, 20472) (429, 1870) (429, 12727) (429, 1)


Stage I      225
Stage II      98
Stage III     70
Stage IV      18
Normal        18
Name: ajcc_pathologic_tumor_stage, dtype: int64

# Log2 transform the mRNA, microRNA, and lncRNA expression values

In [166]:
def expression_val_transform(x):
    return np.log2(x+1)
X_multiomics['GE'] = X_multiomics['GE'].applymap(expression_val_transform)
X_multiomics['MIR'] = X_multiomics['MIR'].applymap(expression_val_transform)
X_multiomics['LNC'] = X_multiomics['LNC'].applymap(expression_val_transform)

# Import Machine Learning Methods

In [167]:
from sklearn import preprocessing
from sklearn import metrics
from sklearn.svm import SVC, LinearSVC
import sklearn.linear_model
from sklearn.model_selection import train_test_split


In [109]:
binarizer = preprocessing.LabelEncoder()
binarizer.fit(y)
binarizer.transform(y)

/opt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([2, 0, 2, 0, 0, 2, 0, 1, 3, 1, 0, 1, 1, 1, 2, 0, 2, 0, 3, 2, 0, 0,
       0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0,
       1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 0, 2, 1, 0, 1, 2, 2, 2, 0, 1, 1,
       2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 3, 0, 2, 2, 2, 0, 0, 0, 0, 1, 0, 3,
       1, 2, 1, 0, 0, 1, 3, 0, 2, 0, 0, 1, 1, 0, 2, 1, 1, 1, 0, 0, 0, 2,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 3,
       1, 0, 0, 2, 0, 1, 3, 0, 1, 2, 0, 0, 0, 0, 0, 0, 2, 1, 1, 2, 2, 2,
       0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 0, 2, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0,
       0, 2, 0, 0, 2, 0, 1, 2, 0, 0, 1, 0, 0, 3, 2, 1, 1, 2, 1, 0, 0, 2,
       0, 3, 2, 0, 3, 1, 0, 0, 1, 3, 2, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 3, 1, 1, 2, 0, 0, 0, 1,

In [181]:
for omic in ["GE", "MIR", "LNC"]:
    print(omic)
    scaler = sklearn.preprocessing.StandardScaler(copy=True, with_mean=True, with_std=False)
    scaler.fit(X_multiomics[omic])

    X_train, X_test, Y_train, Y_test = \
        train_test_split(X_multiomics[omic], y, test_size=0.3, random_state=np.random.randint(0, 10000), stratify=y)
    print(X_train.shape, X_test.shape)
    

    X_train = scaler.transform(X_train)
    
    model = LinearSVC(C=1e-2, penalty='l1', class_weight='balanced', dual=False, multi_class="ovr")
#     model = sklearn.linear_model.LogisticRegression(C=1e-0, penalty='l1', fit_intercept=False, class_weight="balanced")
#     model = SVC(C=1e0, kernel="rbf", class_weight="balanced", decision_function_shape="ovo")

    model.fit(X=X_train, y=Y_train)
    print("NONZERO", len(np.nonzero(model.coef_)[0]))
    print("Training accuracy", metrics.accuracy_score(model.predict(X_train), Y_train))
    print(metrics.classification_report(y_pred=model.predict(X_test), y_true=Y_test))


GE
(300, 20472) (129, 20472)


/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NONZERO 211
Training accuracy 0.77
             precision    recall  f1-score   support

     Normal       1.00      0.80      0.89         5
    Stage I       0.67      0.53      0.59        68
   Stage II       0.39      0.53      0.45        30
  Stage III       0.18      0.14      0.16        21
   Stage IV       0.08      0.20      0.11         5

avg / total       0.51      0.47      0.48       129

MIR
(300, 1870) (129, 1870)
NONZERO 34
Training accuracy 0.57
             precision    recall  f1-score   support

     Normal       0.00      0.00      0.00         5
    Stage I       0.00      0.00      0.00        68
   Stage II       0.00      0.00      0.00        30
  Stage III       0.00      0.00      0.00        21
   Stage IV       0.04      1.00      0.08         5

avg / total       0.00      0.04      0.00       129

LNC
(300, 12727) (129, 12727)


/opt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


NONZERO 21
Training accuracy 0.55
             precision    recall  f1-score   support

     Normal       0.16      1.00      0.27         5
    Stage I       0.55      0.72      0.62        68
   Stage II       0.00      0.00      0.00        30
  Stage III       0.00      0.00      0.00        21
   Stage IV       0.12      0.20      0.15         5

avg / total       0.30      0.43      0.35       129

